---
description: "Logging... logging everywhere! \U0001F52E"
---

# Loggers

Welcome to the _"Logging"_ tutorial of the _"From Zero to Hero"_ series. In this part we will present the functionalities offered by the _Avalanche_ `logging` module.

In [ ]:
!pip install git+https://github.com/ContinualAI/avalanche.git

### 📑 The Logging Module

In the previous tutorial we have learned how to evaluate a continual learning algorithm in _Avalanche_, through different metrics that can be used _off-the-shelf_ via the _Evaluation Plugin_ or stand-alone. However, computing metrics and collecting results, may not be enough at times.

While running complex experiments with long _waiting times_, **logging** results over-time is fundamental to "_babysit_" your experiments in real-time, or even understand what went wrong in the aftermath.

This is why in Avalanche we decided to put a strong emphasis on logging and **provide a number of loggers** that can be used with any set of metrics!

### Loggers

_Avalanche_ at the moment supports three main Loggers:

* **InteractiveLogger**: This logger provides a nice progress bar and displays real-time metrics results in an interactive way \(meant for `stdout`\).
* **TextLogger**: This logger, mostly intended for file logging, is the plain text version of the `InteractiveLogger`. Keep in mind that it may be very verbose.
* **TensorboardLogger**: It logs all the metrics on [Tensorboard](https://www.tensorflow.org/tensorboard) in real-time. Perfect for real-time plotting.

In order to keep track of when each metric value has been logged, we leverage a `global counter`. You can see the `global counter` reported in the x axis of the logged plots.

The `global counter` is an ever-increasing value which starts from 0 and it is increased by one each time a training or evaluation iteration is performed (i.e. after each training or evaluation minibatch).
The `global counter` is updated automatically by the strategy. It should be reset by creating a new instance of the strategy.

#### How to use Them

In [3]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, \
accuracy_metrics, loss_metrics, timing_metrics, cpu_usage_metrics, \
StreamConfusionMatrix, disk_usage_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.strategies import Naive

scenario = SplitMNIST(n_experiences=5)

# MODEL CREATION
model = SimpleMLP(num_classes=scenario.n_classes)

# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(experience=True),
    forgetting_metrics(experience=True, stream=True),
    StreamConfusionMatrix(num_classes=scenario.n_classes, save_image=False),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)

# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = Naive(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=500, train_epochs=1, eval_mb_size=100,
    evaluator=eval_plugin)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(scenario.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 9]
-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --
100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:14<00:00,  1.61it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 720915.6689
	DiskUsage_MB/train_phase/train_stream/Task000 = 720915.3457
	Loss_Epoch/train_phase/train_stream/Task000 = 1.0546
	Loss_MB/train_phase/train_stream/Task000 = 0.2314
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0334
	Time_Epoch/train_phase/train_stream/Task000 = 14.4948
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.7566
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9839
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████████████████████████████████

	DiskUsage_Stream/eval_phase/test_stream = 720954.7568
	Loss_Stream/eval_phase/test_stream = 3.3306
	Top1_Acc_Stream/eval_phase/test_stream = 0.2142
Start of experience:  2
Current Classes:  [5, 6]
-- >> Start of training phase << --
-- Starting training on experience 2 (Task 0) from train stream --
100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:14<00:00,  1.58it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 720963.3389
	DiskUsage_MB/train_phase/train_stream/Task000 = 720963.0156
	Loss_Epoch/train_phase/train_stream/Task000 = 2.4399
	Loss_MB/train_phase/train_stream/Task000 = 0.7766
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0335
	Time_Epoch/train_phase/train_stream/Task000 = 14.1599
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3270
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.8555
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start

	DiskUsage_Stream/eval_phase/test_stream = 720981.1094
	Loss_Stream/eval_phase/test_stream = 2.0952
	Top1_Acc_Stream/eval_phase/test_stream = 0.3063
Start of experience:  4
Current Classes:  [1, 4]
-- >> Start of training phase << --
-- Starting training on experience 4 (Task 0) from train stream --
100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:15<00:00,  1.68it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 720990.6611
	DiskUsage_MB/train_phase/train_stream/Task000 = 720990.3379
	Loss_Epoch/train_phase/train_stream/Task000 = 2.2713
	Loss_MB/train_phase/train_stream/Task000 = 1.1053
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0316
	Time_Epoch/train_phase/train_stream/Task000 = 15.0845
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.2687
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9048
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start

### Create your Logger

If the available loggers are not sufficient to suit your needs, you can always implement a custom logger by specializing the behaviors of the `StrategyLogger` base class.

This completes the "_Logging_" tutorial for the "_From Zero to Hero_" series. We hope you enjoyed it!

## 🤝 Run it on Google Colab

You can run _this chapter_ and play with it on Google Colaboratory: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ContinualAI/colab/blob/master/notebooks/avalanche/loggers.ipynb)